In [1]:
import pyspark as sp
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,date_format,concat,substring,lit,lpad,to_timestamp,when
import  pandas as pd
import os

In [2]:
spark = SparkSession.builder \
    .appName("PreProcessing") \
    .getOrCreate()

24/04/18 10:44:16 WARN Utils: Your hostname, Subins-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.65.210.91 instead (on interface en0)
24/04/18 10:44:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/18 10:44:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
for filename in os.listdir("./dataConverted"):
    #   entry=os.path.join("./dataConverted",dir)  
    #   if os.path.isdir(entry):
            #for filename in os.listdir(entry):
                  df=spark.read.option("header","true").option("inferSchema", "true").csv("./dataConverted/"+filename)
                  df = df.withColumn("lat", df["lat"] / 1000)
                  df = df.withColumn("long", df["long"] / 1000)
                  df = df.withColumn("airtemp", df["airtemp"] / 10)
                  df = df.withColumn("dewpointtemp", df["dewpointtemp"] / 10)
                  df = df.withColumn("atmpress", df["atmpress"] / 10)
                  df = df.withColumn("date", 
                   concat(substring(col("date"), 1, 4), 
                          lit("-"), 
                          substring(col("date"), 5, 2), 
                          lit("-"), 
                          substring(col("date"), 7, 2)))
                  df = df.withColumn("time", lpad(col("time"), 4, "0"))
                  df = df.withColumn("time", 
                   concat(substring(col("time"), 1, 2), 
                          lit(":"),
                          substring(col("time"), 3, 2)))
                  df = df.filter(df["airtemp"] != 999.9)
                  pd_df=df.toPandas()
                  #df.coalesce(1).write.csv("./dataPreprocessed/"+dir+'/'+dir+'.csv',mode="overwrite", header=True)
                  pd_df.to_csv('./dataPreprocessed/'+filename, index=False,mode='a')